In [1]:
import argparse
import os
import numpy as np
import ray

In [18]:
#!time python train_centralised.py

In [3]:
!time python rllib_test.py

2023-02-10 10:01:38,360	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(ApexDDPG pid=30901) 2023-02-10 10:01:48,720	WARNING algorithm_config.py:488 -- Cannot create ApexDDPGConfig from given `config_dict`! Property __stdout_file__ not supported.
(ApexDDPG pid=30901) 2023-02-10 10:01:49,375	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(ApexDDPG pid=30901) 2023-02-10 10:01:54,446	ERROR actor_manager.py:486 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=30910, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f9ee8a44790>)
(ApexDDPG pid=30901)   File "/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 712, in __init__
(ApexDDPG pid=30901)     self._build_policy_ma

2023-02-10 10:01:54,555	ERROR trial_runner.py:1088 -- Trial APEX_DDPG_waterworld_eadb0_00000: Error processing event.
ray.tune.error._TuneNoNextExecutorEventError: Traceback (most recent call last):
  File "/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/ray/tune/execution/ray_trial_executor.py", line 1070, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/ray/_private/worker.py", line 2311, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::ApexDDPG.__init__() (pid=30901, ip=127.0.0.1, repr=ApexDDPG)
  File "/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/ray/rllib/evaluation/worker_set.py", line 239, in _setup
    self.add_workers(
  File 

(ApexDDPG pid=30901) 2023-02-10 10:01:54,501	ERROR worker.py:763 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::ApexDDPG.__init__() (pid=30901, ip=127.0.0.1, repr=ApexDDPG)
(ApexDDPG pid=30901)   File "/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/ray/rllib/evaluation/worker_set.py", line 239, in _setup
(ApexDDPG pid=30901)     self.add_workers(
(ApexDDPG pid=30901)   File "/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/ray/rllib/evaluation/worker_set.py", line 612, in add_workers
(ApexDDPG pid=30901)     raise result.get()
(ApexDDPG pid=30901)   File "/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/ray/rllib/utils/actor_manager.py", line 473, in __fetch_result
(ApexDDPG pid=30901)     result = ray.get(r)
(ApexDDPG pid=30901) ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=30910, ip=127.0.0.1, repr=<

In [20]:
from gymnasium.spaces import Box, Discrete
from ray import air, tune
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.rllib.algorithms.dqn import DQNConfig
from ray.rllib.env import PettingZooEnv
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import register_env
from src.envs import antisocial_ring_v0

In [21]:
from ray.tune.logger import pretty_print

In [22]:
env = ParallelPettingZooEnv(antisocial_ring_v0.parallel_env())
print(env.reset())
print(env.action_space)
env_name = "AntisocialRing"
register_env(env_name, lambda config: ParallelPettingZooEnv(antisocial_ring_v0.parallel_env()))

{'0': array([1., 1., 0.]), '1': array([1., 1., 0.]), '2': array([0., 1., 0.]), '3': array([0., 1., 1.]), '4': array([0., 1., 1.])}
Discrete(3)


/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/gymnasium/spaces/box.py:129: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [34]:
from ray.rllib.env import PettingZooEnv
from pettingzoo.classic import rps_v2
from ray.rllib.algorithms.pg import (
    PG,
    PGConfig,
    PGTF2Policy,
    PGTF1Policy,
    PGTorchPolicy,
)
from ray.rllib.utils.test_utils import check_learning_achieved

In [36]:
ray.shutdown()

In [37]:
def env_creator(args):
    env = rps_v2.env()
    return env


register_env("RockPaperScissors", lambda config: PettingZooEnv(env_creator(config)))

ray.init()

stop = {
        "training_iteration": 150,
        "timesteps_total": 100,
        "episode_reward_mean": 100,
    }

config = PGConfig().environment("RockPaperScissors").framework('torch')

results = tune.Tuner(
     "PG", param_space=config, run_config=air.RunConfig(stop=stop, verbose=1)).fit()



(PG pid=27168) 2023-02-09 15:16:30,965	WARNING algorithm_config.py:488 -- Cannot create PGConfig from given `config_dict`! Property __stdout_file__ not supported.
(PG pid=27168) 2023-02-09 15:16:33,133	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(PG pid=27168) 2023-02-09 15:16:33,137	WARNING env.py:51 -- Skipping env checking for this experiment
(PG pid=27168) 2023-02-09 15:16:33,182	ERROR worker.py:763 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PG.__init__() (pid=27168, ip=127.0.0.1, repr=PG)
(PG pid=27168)   File "/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/ray/rllib/algorithms/algorithm.py", line 441, in __init__
(PG pid=27168)     super().__init__(
(PG pid=27168)   File "/Users/Katie/code/eco_rl_games/env/lib/python3.8/site-packages/ray/tune/trainable/trainable.py", line 169, in __init__
(PG pid=27168)     se